In [100]:
import pandas as pd
import os
import torch
print(torch.__version__)


2.0.1


In [101]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges


In [102]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

/Users/bargav/anaconda3/envs/clean_ml/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [103]:
data = dataset.data
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [104]:
data.edge_index[1]

tensor([ 628,  158,  486,  ..., 2820, 1643,   33])

In [105]:
data = train_test_split_edges(data)


/Users/bargav/anaconda3/envs/clean_ml/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [106]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

In [107]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels):
        super(GCNEncoder, self).__init__()
        self.gconv1 = GCNConv(in_channels, hidden_dim, cached=True) # Cached for transductive learning
        self.gconv2 = GCNConv(hidden_dim, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.gconv1(x, edge_index).relu()
        return self.gconv2(x, edge_index)

In [108]:
data.x.shape[1]

3703

In [109]:
in_channels = data.x.shape[1]
out_channels = 2
hidden_dim = 24

num_epochs = 100

encoder = GCNEncoder(in_channels,hidden_dim, out_channels)



In [110]:
# Load available Device 

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mps_device = torch.device('mps')

In [111]:
device = mps_device

In [112]:

# Graph AutoEncoder

from torch_geometric.nn import GAE

model = GAE(encoder)

model = model.to(device)

In [113]:
X = data.x.to(device)

train_pos_edge_indices = data.train_pos_edge_index.to(device)

In [114]:
# Optimizer

optimizer = torch.optim.Adam(model.parameters(), lr  = 0.001)

In [115]:
# Using Tensorboard

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f'runs/GAE_experiment_{out_channels}d_{num_epochs}epochs')


In [116]:
def train():
    model.train()

    optimizer.zero_grad()
    z = model.encode(X, train_pos_edge_indices)
    loss = model.recon_loss(z, train_pos_edge_indices)

    loss.backward()
    optimizer.step()
    return loss

def test(pos_edge_indices, neg_edge_indices):
    model.eval()

    with torch.no_grad():
        z = model.encode(X, train_pos_edge_indices)
        
    return model.test(z, pos_edge_indices, neg_edge_indices)




In [117]:
for epoch in range(num_epochs):

    loss = train()

    auc, avg_precision = test(data.test_pos_edge_index, data.test_neg_edge_index)

    print(f"Epoch {epoch:03d}, AUC: {auc:.3f}, Average Precision: {avg_precision:.3f}")

    writer.add_scalar('auc train', auc, epoch)
    writer.add_scalar('average precision', avg_precision, epoch )



Epoch 000, AUC: 0.625, Average Precision: 0.635
Epoch 001, AUC: 0.640, Average Precision: 0.664
Epoch 002, AUC: 0.649, Average Precision: 0.681
Epoch 003, AUC: 0.655, Average Precision: 0.694
Epoch 004, AUC: 0.662, Average Precision: 0.704
Epoch 005, AUC: 0.668, Average Precision: 0.712
Epoch 006, AUC: 0.678, Average Precision: 0.723
Epoch 007, AUC: 0.691, Average Precision: 0.736
Epoch 008, AUC: 0.707, Average Precision: 0.749
Epoch 009, AUC: 0.726, Average Precision: 0.764
Epoch 010, AUC: 0.747, Average Precision: 0.779
Epoch 011, AUC: 0.769, Average Precision: 0.795
Epoch 012, AUC: 0.788, Average Precision: 0.808
Epoch 013, AUC: 0.802, Average Precision: 0.819
Epoch 014, AUC: 0.811, Average Precision: 0.826
Epoch 015, AUC: 0.818, Average Precision: 0.832
Epoch 016, AUC: 0.823, Average Precision: 0.835
Epoch 017, AUC: 0.826, Average Precision: 0.838
Epoch 018, AUC: 0.829, Average Precision: 0.840
Epoch 019, AUC: 0.831, Average Precision: 0.841
Epoch 020, AUC: 0.833, Average Precision

In [118]:
Z = model.encode(X, train_pos_edge_indices)
print(Z)

tensor([[-0.3539,  0.4916],
        [ 1.3507,  0.1492],
        [-0.9123,  0.0574],
        ...,
        [ 0.8074,  1.0721],
        [-0.8290, -0.1964],
        [-0.6791,  0.7380]], device='mps:0', grad_fn=<AddBackward0>)
